<a href="https://colab.research.google.com/github/Ajoydey/ICICV_conference/blob/main/Markov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pymaxflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 123 kB 3.2 MB/s 
  Created wheel for pymaxflow: filename=PyMaxflow-1.2.13-cp37-cp37m-linux_x86_64.whl size=529501 sha256=438edf993754099f50fcfc9e5e8d26682fb585fa8884007ee2086b543549795c
  Stored in directory: /root/.cache/pip/wheels/62/f9/eb/62e4c1fcbee43e64b749674879fd3248d8c49f08c37c8a564d
Successfully built pymaxflow


In [ ]:
import numpy as np
import maxflow as mf
from PIL import Image
import cv2
import scipy
import scipy.ndimage
import threading
import copy
from sklearn.cluster import KMeans
import math

In [ ]:
#For helping in multi threading

from threading import Thread

class ThreadWithReturnValue(Thread):
    def __init__(self, group=None, target=None, name=None,
                 args=(), kwargs={}, Verbose=None):
        Thread.__init__(self, group, target, name, args, kwargs)
        self._return = None
    def run(self):
        print(type(self._target))
        if self._target is not None:
            self._return = self._target(*self._args,
                                                **self._kwargs)
    def join(self, *args):
        Thread.join(self, *args)
        return self._return

In [ ]:
def calculate_energy(img_orig, img_work):
    '''Calculates Energy of image.
       img: is input array'''

    m, n = img_orig.shape
    E_data = 0
    for i in range(m):
        for j in range(n):
            E_data += D_p(img_orig[i][j], img_work, j, i)
    
    E_smooth = 0
    for i in range(m):
        for j in range(n):
            ns = give_neighbours(img_work, j, i)
            E_smooth += sum([V_p_q(v, img_work[i][j]) for v in ns])

    return E_data + E_smooth

def V_p_q(label1, label2, w1=10):
    '''Definition of the potential'''
    return min(w1*(label1 - label2)**2, math.log((label1 - label2)**2+1))
    


def D_p(label, graph, x, y, w=0.2):
    return w * math.log((label -graph[y][x])**2 +1) #best working D_p
    


def give_neighbours(image, x, y):
    '''Returns a list of all neighbour intensities'''
    m,n = image.shape
    if x>=n or x<0 or y>=m or y<0:
       raise ValueError('Pixel is not in image. x and/or y are to large')
    ns = []
    for a,b in zip([1,0,-1,0],[0,1,0,-1]):
        if (x+a<n and x+a>=0) and (y+b<m and y+b>=0):
            ns.append(image[y+b][x+a])
    return ns 

def return_mapping_of_image(image, alpha, beta):
    #map does the position in graph map to (y,x) position in image
    m,n = image.shape
    map = {}
    #other way 
    revmap = {}
    #loop over all pixels and add them to maps
    map_parameter = 0
    for y in range(m):
        for x in range(n):
            #extract pixel which have the wanted label
            if image[y][x] == alpha or image[y][x] == beta:
                map[map_parameter] = (y,x)
                revmap[(y,x)] = map_parameter
                map_parameter += 1
    
    return map, revmap


def alpha_beta_swap_new(alpha, beta, img_orig, img_work):
    ''' Performs alpha-beta-swap
        img_orig: input image 
        img_work: denoised image in each step
        time_measure: flag if you want measure time'''

    #extract position of alpha or beta pixels to mapping 
    map, revmap = return_mapping_of_image(img_work, alpha, beta)
    
    #graph of maxflow 
    graph_mf = mf.Graph[float](len(map) )
    #add nodes
    nodes = graph_mf.add_nodes(len(map))
            
    #add n-link edges
    weight = V_p_q(alpha, beta)
    for i in range(0,len(map)):
        y,x = map[i]
        #top, left, bottom, right
        for a,b in zip([1,0,-1,0],[0,1,0,-1]):
            if (y+b, x+a) in revmap:
                graph_mf.add_edge(i,revmap[(y+b,x+a)], weight, 0)
   
    #add all the terminal edges
    for i in range(0,len(map)):
        y,x = map[i]
        #find neighbours
        neighbours = give_neighbours(img_work, x, y)
        #consider only neighbours which are not having alpha or beta label
        fil_neigh = list(filter(lambda i: i!=alpha and i!=beta, neighbours))
        #calculation of weight
        t_weight_alpha = sum([V_p_q(alpha,v) for v in fil_neigh]) + D_p(alpha, img_orig, x, y)
        t_weight_beta = sum([V_p_q(beta,v) for v in fil_neigh]) + D_p(beta, img_orig, x, y)
        graph_mf.add_tedge(nodes[i], t_weight_alpha, t_weight_beta)

    #calculating flow
    flow = graph_mf.maxflow()
    res = [graph_mf.get_segment(nodes[i]) for i in range(0, len(nodes))]
    
    #depending on cut assign new label
    for i in range(0, len(res)):
        y, x = map[i] 
        if res[i] == 1:
            img_work[y][x] = alpha 
        else:
            img_work[y][x] = beta
    
    return img_work


def swap_minimization(img_orig, img_work, labels, cycles = 10):
    '''This methods implements the energy minimization via alpha-beta-swaps
       img_orig: is original input image
       img_work: optimized image
       cycles: how often to iterate over all labels'''
    m,n = img_orig.shape
    #do iteration of all pairs a few times
    for u in range(0,cycles):
        # shuffle(labels)
        #iterate over all pairs of labels0
        for i in range(len(labels)):
            for j in range(i+1, len(labels)):
                #computing intensive swapping and graph cutting part
                img_work  = alpha_beta_swap_new(labels[i],labels[j], img_orig, img_work)     
        
        print (str(u+1) + "\t\t\t", calculate_energy(img_orig, img_work)) 
    
    return img_work

In [ ]:
R, G, B = 0, 1, 2  # index for convenience
L = 256  # color depth


def get_dark_channel(I, w):
    """Get the dark channel prior in the (RGB) image data.
    Parameters
    -----------
    I:  an M * N * 3 numpy array containing data ([0, L-1]) in the image where
        M is the height, N is the width, 3 represents R/G/B channels.
    w:  window size
    Return
    -----------
    An M * N array for the dark channel prior ([0, L-1]).
    """
    M, N, _ = I.shape
    padded = np.pad(I, ((w // 2, w // 2), (w // 2, w // 2), (0, 0)), 'edge')
    darkch = np.zeros((M, N))
    for i, j in np.ndindex(darkch.shape):
        darkch[i, j] = np.min(padded[i:i + w, j:j + w, :])  # CVPR09, eq.5
    return darkch


def get_atmosphere(I, darkch, p):
    """Get the atmosphere light in the (RGB) image data.
    Parameters
    -----------
    I:      the M * N * 3 RGB image data ([0, L-1]) as numpy array
    darkch: the dark channel prior of the image as an M * N numpy array
    p:      percentage of pixels for estimating the atmosphere light
    Return
    -----------
    A 3-element array containing atmosphere light ([0, L-1]) for each channel
    """
    # reference CVPR09, 4.4
    M, N = darkch.shape
    flatI = I.reshape(M * N, 3)
    flatdark = darkch.ravel()
    searchidx = (-flatdark).argsort()[:(int)(M * N * p)]  # find top M * N * p indexes
    print ('atmosphere light region:', [(i // N, i % N) for i in searchidx])

    # return the highest intensity for each channel
    return np.max(flatI.take(searchidx, axis=0), axis=0)

def get_transmission(I, r):
    

    hsvI = cv2.cvtColor(I, cv2.COLOR_BGR2HSV)
    s = hsvI[:,:,1] / 255.0
    v = hsvI[:,:,2] / 255.0

    #sigma = 0.041337
    sigma = 0.05
    sigmaMat = np.random.normal(0, sigma, (I.shape[0], I.shape[1]))

    output =  0.121779 + 0.959710 * v - 0.780245 * s + sigmaMat
    #output =  0.12 + 0.96 * v - 0.78 * s + sigmaMat
    outputPixel = output
    output = scipy.ndimage.filters.minimum_filter(output,(r,r))
    rawt = output
    #cv2.imwrite("data/vsFeature.jpg", outputRegion*255 )
    labels, labelled_px = K_Mean(rawt, k=24)
    refinedt = np.array([labels[x] for x in labelled_px]).reshape(rawt.shape)
    return rawt, refinedt, labels

def K_Mean(t, k=10):
  Kmean = KMeans(n_clusters = k)

  M,N =  t.shape
  img1 = t.reshape(M*N,1)
  Kmean.fit(img1)
  return Kmean.cluster_centers_, Kmean.labels_

def dehaze_raw(I, im, tmin=0.2, Amax=220, w=15, p=0.001,
               omega=0.95, guided=True, r=40, eps=1e-3):  #guided changed to False p changed to0.001
    """Get the dark channel prior, atmosphere light, transmission rate
       and refined transmission rate for raw RGB image data.
    Parameters
    -----------
    I:      M * N * 3 data as numpy array for the hazy image
    tmin:   threshold of transmission rate
    Amax:   threshold of atmosphere light
    w:      window size of the dark channel prior
    p:      percentage of pixels for estimating the atmosphere light
    omega:  bias for the transmission estimate
    guided: whether to use the guided filter to fine the image
    r:      the radius of the guidance
    eps:    epsilon for the guided filter
    Return
    -----------
    (Idark, A, rawt, refinedt) if guided=False, then rawt == refinedt
    """
    m, n, _ = I.shape
    Idark = get_dark_channel(I, w)

    thread1 = ThreadWithReturnValue(target = get_atmosphere, args=(I, Idark, p,))
    thread2 = ThreadWithReturnValue(target = get_transmission, args =(im, r,))
    thread2.start()
    thread1.start()
    A = thread1.join()
    rawt, refinedt, labels = thread2.join()   #t from color attenuation prior
    A = np.minimum(A, Amax)  # threshold A
    print ('atmosphere', A)

    rawt = np.maximum(rawt, tmin)  # threshold t
    
    
    print ("Initial energy-",calculate_energy(rawt, refinedt) )
    swap_minimization(rawt, refinedt, labels, cycles=3)

    
    print ('raw transmission rate between [%.4f, %.4f]' % (rawt.min(), rawt.max()))
    
    print ('refined transmission rate between [%.4f, %.4f]' % (refinedt.min(), refinedt.max()))

    return Idark, A, rawt, refinedt

def recover(imgar, A, t, tmin=0.1):
    """
    Radiance recovery. According to section (4.3) and equation (16) in the reference paper
    http://kaiminghe.com/cvpr09/index.html

    Parameters
    -----------
    imgar:    an H*W RGB hazed image
    atm:      the atmospheric light in imgar
    t:        the transmission in imgar
    tmin:     the minimum value that transmission can take (default=0.1)

    Return
    -----------
    The imaged recovered and dehazed, j (a H*W RGB matrix).
    """
    epsilon = 0.000001
    delta= 0.15

    # the output dehazed image

    j = np.zeros(imgar.shape)
    Transmission = pow(np.maximum(abs(t), epsilon), delta)

    # equation (16)

    HazeCorrectedImage = copy.deepcopy(imgar)
    if (len(imgar.shape) == 3):
        for ch in range(len(imgar.shape)):
            temp = ((imgar[:, :, ch].astype(float) - A[ch]) / Transmission) + A[ch]
            temp = np.maximum(np.minimum(temp, 255), 0)
            HazeCorrectedImage[:, :, ch] = temp
    else:
        temp = ((imgar.astype(float) - A[0]) / Transmission) + A[0]
        temp = np.maximum(np.minimum(temp, 255), 0)
        HazeCorrectedImage = temp
    return (HazeCorrectedImage)

def dehaze(im, im1, tmin=0.2, Amax=220, w=15, p=0.001,
           omega=0.95, guided=True, r=40, eps=1e-3):  #guided changed to false  p changed to 0.001
    """Dehaze the given RGB image.
    Parameters
    ----------
    im:     the Image object of the RGB image
    guided: refine the dehazing with guided filter or not
    other parameters are the same as `dehaze_raw`
    Return
    ----------
    (dark, rawt, refinedt, rawrad, rerad)
    Images for dark channel prior, raw transmission estimate,
    refiend transmission estimate, recovered radiance with raw t,
    recovered radiance with refined t.
    """
    I = np.asarray(im, dtype=np.float64)
    Idark, A, rawt, refinedt = dehaze_raw(I, im1, tmin, Amax, w, p,
                                          omega, guided, r, eps)
    white = np.full_like(Idark, L - 1)

    def to_img(raw):
        # threshold to [0, L-1]
        cut = np.maximum(np.minimum(raw, L - 1), 0).astype(np.uint8)

        if len(raw.shape) == 3:
            print ('Range for each channel:')
            for ch in range(3):
                print ('[%.2f, %.2f]' % (raw[:, :, ch].max(), raw[:, :, ch].min()))
            return Image.fromarray(cut)
        else:
            return Image.fromarray(cut)

    return [to_img(raw) for raw in (Idark, white * rawt, white * refinedt,
                                    recover(I, A, rawt),
                                    recover(I, A, refinedt))]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import argparse
from functools import partial
import time
import cv2
from PIL import Image
import scipy
import scipy.ndimage



SP_IDX = (5, 6, 8, 12)  # for testing parameters
SP_PARAMS = ({'tmin': 0.2, 'Amax': 170, 'w': 15, 'r': 40},
             {'tmin': 0.5, 'Amax': 190, 'w': 15, 'r': 40},
             {'tmin': 0.5, 'Amax': 220, 'w': 15, 'r': 40})


def generate_results(src, dest, generator):
    print ('processing', src + '...')
    im = Image.open(src)
    im1 = cv2.imread(src)
    dark, rawt, refinedt, rawrad, rerad = generator(im, im1)
    #dark.save(dest % 'dark')
    #rawt.save(dest % 'rawt')
    #refinedt.save(dest % 'refinedt')
    #rawrad.save(dest % 'radiance-rawt')
    rerad.save(dest % 'Markov')
    print ('saved', dest)


def main():
    l = ["37","43","45"]
    for list_item in l:
      src = "/content/drive/MyDrive/dehazed pic/"+list_item+"_hazy.png"
      #folder = "/content/drive/MyDrive/dehazed pic"
      dest = "/content/drive/MyDrive/dehazed pic/markov/"+list_item+"_" +"%s" +".jpg"
      t1 = time.perf_counter()
      generate_results(src, dest, dehaze)
      t2 = time.perf_counter()
      print('time taken', t2-t1)

if __name__ == '__main__':
    main()

processing /content/drive/MyDrive/dehazed pic/37_hazy.png...
<class 'function'>
<class 'function'>
atmosphere light region: [(26, 0), (28, 0), (27, 0), (23, 0), (24, 0), (25, 0), (29, 0), (145, 1575), (145, 1576), (147, 1575), (147, 1576), (146, 1576), (146, 1575), (147, 1577), (147, 1578), (146, 1577), (147, 1580), (144, 1575), (147, 1579), (145, 1577), (22, 0), (141, 1575), (141, 1576), (132, 1575), (133, 1575), (136, 1575), (142, 1576), (142, 1575), (140, 1576), (135, 1575), (137, 1575), (147, 1581), (140, 1575), (134, 1575), (143, 1576), (146, 1578), (139, 1575), (138, 1575), (143, 1575), (144, 1576), (51, 1105), (51, 1106), (51, 1107), (51, 1108), (61, 1103), (59, 1103), (51, 1104), (50, 1106), (51, 1112), (51, 1113), (51, 1114), (51, 1115), (51, 1116), (54, 1102), (51, 1111), (51, 1110), (51, 1103), (54, 1103), (50, 1105), (50, 1107), (60, 1103), (60, 1102), (50, 1104), (56, 1102), (51, 1102), (56, 1103), (50, 1102), (49, 1103), (57, 1103), (57, 1102), (49, 1102), (50, 1108), (50